In [1]:
import re, time, json, os
from dataclasses import dataclass
from pathlib import Path
from typing import List, Tuple, Optional, Iterable, Dict

In [2]:
# ----------------------------
# Paths (EDIT THESE)
# ----------------------------
DOCS_ROOT = Path("/Users/clockorangezoe/Documents/phd_projects/code/geoAI/RDProLLMagent/doc/docs")
PY_PATH   = Path("/Users/clockorangezoe/Documents/phd_projects/code/geoAI/RDProLLMagent/python/ndvi.py")

assert DOCS_ROOT.exists(), f"Missing docs root: {DOCS_ROOT}"
assert PY_PATH.exists(), f"Missing python script: {PY_PATH}"

py_code = PY_PATH.read_text(encoding="utf-8")


In [3]:
# ----------------------------
# Markdown chunking utilities
# ----------------------------
@dataclass
class Chunk:
    chunk_id: str
    heading_path: str
    level: int
    text: str

def chunk_markdown_by_headings(md: str) -> List[Chunk]:
    lines = md.splitlines()
    starts = []
    for i, line in enumerate(lines):
        m = re.match(r"^(#{2,3})\s+(.*\S)\s*$", line)
        if m:
            starts.append((i, len(m.group(1)), m.group(2).strip()))

    if not starts:
        return [Chunk("c0000", "DOC", 1, md.strip() + "\n")]

    chunks: List[Chunk] = []
    current_h2: Optional[str] = None

    for idx, (start_i, level, title) in enumerate(starts):
        end_i = starts[idx + 1][0] if idx + 1 < len(starts) else len(lines)
        body = "\n".join(lines[start_i:end_i]).strip() + "\n"

        if level == 2:
            current_h2 = title
            heading_path = current_h2
        else:
            heading_path = f"{current_h2} / {title}" if current_h2 else title

        chunks.append(Chunk(f"c{idx:04d}", heading_path, level, body))

    return chunks

def make_doc_pack(selected: List[Chunk]) -> str:
    out = []
    for ch in selected:
        out.append(f"### DOC CHUNK {ch.chunk_id}: {ch.heading_path}\n{ch.text}\n")
    return "\n".join(out).strip() + "\n"

In [4]:
# ----------------------------
# Docs loader: key -> .md file(s)
# ----------------------------
def load_docs_for_keys_with_aliases(
    root: Path,
    keys: Iterable[str],
    *,
    encoding: str = "utf-8",
    aliases: Optional[Dict[str, str]] = None,
) -> List[Tuple[Path, str]]:
    """
    keys: things you care about (API names or concepts).
    aliases: maps key -> filename_stem (e.g., 'mapPixels' -> 'mappixels').
    Returns unique docs in first-seen order.
    """
    if not root.exists():
        raise FileNotFoundError(f"DOCS_ROOT does not exist: {root}")

    aliases = aliases or {}

    def norm(s: str) -> str:
        return s.strip().lower()

    # index: stem -> path
    md_index: Dict[str, Path] = {}
    for p in root.rglob("*.md"):
        md_index[norm(p.stem)] = p

    resolved_paths: List[Path] = []
    seen = set()

    for k in keys:
        stem = norm(aliases.get(k, k))  # apply alias if any

        # direct stem match
        path = md_index.get(stem)

        # fallback: normalize to alnum only (handles camelCase-ish inputs)
        if path is None:
            stem2 = "".join(ch for ch in stem if ch.isalnum())
            path = md_index.get(stem2)

        if path is None:
            available = sorted(md_index.keys())
            raise FileNotFoundError(
                f"No .md doc for key '{k}' (resolved stem '{stem}') under {root}\n"
                f"Available docs: {available}"
            )

        if path not in seen:
            resolved_paths.append(path)
            seen.add(path)

    return [(p, p.read_text(encoding=encoding)) for p in resolved_paths]

In [5]:
MODEL = "gpt-5"

SYSTEM_PROMPT = """
You are a geospatial data engineer and Spark systems expert.

Task: Convert a given geospatial Python script into Scala code that runs on RDPro (Spark-based raster processing) on Apache Spark.

You must understand Spark execution and produce distributed, RDD-based Scala.

Environment & paths:
- Determine whether output paths should be treated as local or distributed based on Spark configuration and the URI scheme.
- You MAY use standard Spark/Scala APIs for this (SparkConf, SparkContext.hadoopConfiguration, java.net.URI, java.nio.file).
- You MUST NOT invent any RDPro path utilities.

Hard rules:
1) Output MUST be valid Scala that compiles as a Spark job (a complete file). Include:
   - necessary imports
   - a runnable entrypoint: `object JobName { def main(args:Array[String]):Unit = ... }` (or `extends App`)
   - SparkSession initialization
   - spark.stop() at the end (in finally or equivalent)
2) Use ONLY RDPro APIs that appear in the provided DOC CHUNKS.
   - If a method signature is not shown in DOC CHUNKS, do NOT guess.
3) Do NOT invent RDPro APIs, overloads, implicits, or helper utilities. No hidden "magic" conversions.
4) Preserve semantics of the Python: raster IO, pixel math, focal ops, masking/nodata, reprojection/resample if present.
5) Distributed correctness:
   - Avoid driver-side operations: do NOT call collect/toLocalIterator unless required by the Python semantics.
   - Prefer RDPro RasterRDD end-to-end when available in DOC CHUNKS.
6) Raster alignment robustness:
   - If not in DOC CHUNKS, fail fast: throw a clear runtime error explaining alignment is required but unsupported with available APIs.
7) Performance guidance (Spark-level only):
   - You MAY set Spark SQL / Spark configs and use standard Spark operations (repartition/coalesce/cache/persist) ONLY when:
     (a) it does not change semantics, and
     (b) it is justified by an obvious pipeline boundary (e.g., before a wide op / expensive reuse).
8) Lambdas:
   - When passing lambdas to RDPro functions (e.g., mapPixels), add explicit parameter and return types so Scala compiles.
9) CLI args:
   - If the Python has input/output paths, read them from args with safe defaults and validation.
   - Do not introduce extra parameters not implied by the Python.

Output format (strict):
- First: Scala file content only (NO markdown fences).
- After the Scala: a "NOTES" section listing:
  (a) RDPro APIs used (names only)
  (b) Unsupported operations and why (especially if missing alignment/warp APIs)
  (c) Assumptions about IO paths / bands / nodata / CRS / environment detection logic
""".strip()

In [6]:
def build_user_prompt(doc_pack: str, py_code: str) -> str:
    return f"""
RDPro documentation (relevant DOC CHUNKS only):
{doc_pack}

Python script:
{py_code}

Task:
Translate the Python script into Scala targeting RDPro on Spark.
Use ONLY APIs described in the DOC CHUNKS.
""".strip()

In [7]:
# ----------------------------
# Manual "keys" (what you used before)
# These are NOT necessarily filenames; aliases map them to doc files.
# ----------------------------
NDVI_KEYS = [
    "datamodel",
    "setup",
    "dataloading",
    "rastermetadata",
    "overlay",
    "mapPixels",
    "saveAsGeoTiff",
    "GeoTiffWriter",
    "Compression",
]

ALIASES = {
    # docs tree uses lowercase stems like mappixels.md
    "mapPixels": "mappixels",

    # concepts that live inside these docs (not separate files)
    "saveAsGeoTiff": "rasterwriting",
    "GeoTiffWriter": "rasterwriting",
    "Compression": "rasterwriting",

    # "geoTiff" read API is typically documented in dataloading
    "geoTiff": "dataloading",
}


In [8]:
# ----------------------------
# Load docs -> chunk -> pack -> final prompt
# ----------------------------
doc_files = load_docs_for_keys_with_aliases(DOCS_ROOT, NDVI_KEYS, aliases=ALIASES)

print("Loaded docs:")
for p, _ in doc_files:
    print(" -", p)

all_chunks: List[Chunk] = []
for path, text in doc_files:
    chunks = chunk_markdown_by_headings(text)
    all_chunks.extend(chunks)

doc_pack = make_doc_pack(all_chunks)
user_prompt = build_user_prompt(doc_pack, py_code)


Loaded docs:
 - /Users/clockorangezoe/Documents/phd_projects/code/geoAI/RDProLLMagent/doc/docs/common/datamodel.md
 - /Users/clockorangezoe/Documents/phd_projects/code/geoAI/RDProLLMagent/doc/docs/common/setup.md
 - /Users/clockorangezoe/Documents/phd_projects/code/geoAI/RDProLLMagent/doc/docs/data/dataloading.md
 - /Users/clockorangezoe/Documents/phd_projects/code/geoAI/RDProLLMagent/doc/docs/common/rastermetadata.md
 - /Users/clockorangezoe/Documents/phd_projects/code/geoAI/RDProLLMagent/doc/docs/process/overlay.md
 - /Users/clockorangezoe/Documents/phd_projects/code/geoAI/RDProLLMagent/doc/docs/process/mappixels.md
 - /Users/clockorangezoe/Documents/phd_projects/code/geoAI/RDProLLMagent/doc/docs/data/rasterwriting.md


In [9]:
# ----------------------------
# Save prompt + doc selection (optional)
# ----------------------------
OUT_DIR = Path("./runs/workspace")
OUT_DIR.mkdir(parents=True, exist_ok=True)

(OUT_DIR / "prompt_manual_multi.txt").write_text(user_prompt, encoding="utf-8")
(OUT_DIR / "doc_selection_multi.json").write_text(
    json.dumps([str(p) for p, _ in doc_files], indent=2),
    encoding="utf-8",
)

print("\nPrompt chars:", len(user_prompt))
print("Saved:", OUT_DIR / "prompt_manual_multi.txt")



Prompt chars: 11393
Saved: runs/workspace/prompt_manual_multi.txt


In [10]:
import os

if "OPENAI_API_KEY" in os.environ:
    print("OPENAI_API_KEY is set")
else:
    print("OPENAI_API_KEY is NOT set")

OPENAI_API_KEY is set


In [11]:
from openai import OpenAI
client = OpenAI()

def run_llm(prompt: str) -> Tuple[str, float]:
    t0 = time.time()
    resp = client.responses.create(
        model=MODEL,
        input=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": prompt},
        ],
    )
    return resp.output_text.strip(), time.time() - t0

scala_out, dt = run_llm(user_prompt)
(OUT_DIR / "Job.manual.scala").write_text(scala_out, encoding="utf-8")

print("Wrote:", OUT_DIR / "ndvi_doc_multi_GPT.scala")
print("LLM latency:", round(dt, 2), "s")
print("\n--- Preview ---\n", scala_out[:900])

Wrote: runs/workspace/ndvi_doc_multi_GPT.scala
LLM latency: 143.63 s

--- Preview ---
 import org.apache.spark.sql.SparkSession
import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.rdd.RDD
import edu.ucr.cs.bdlab.beast._
import java.net.URI
import java.nio.file.{Files, Paths, Path}

object ComputeNdviRDPro {
  def main(args: Array[String]): Unit = {
    val defaultB4 = "/Users/clockorangezoe/Documents/phd_projects/code/geoAI/RDProLLMagent/data/landsat8/LA/B4/LC08_L2SP_040037_20250827_20250903_02_T1_SR_B4.TIF"
    val defaultB5 = "/Users/clockorangezoe/Documents/phd_projects/code/geoAI/RDProLLMagent/data/landsat8/LA/B5/LC08_L2SP_040037_20250827_20250903_02_T1_SR_B5.TIF"
    val defaultOut = "ndvi.tif"

    val b4Path = if (args.length >= 1) args(0) else defaultB4
    val b5Path = if (args.length >= 2) args(1) else defaultB5
    val outPath = if (args.length >= 3) args(2) else defaultOut

    val spark = SparkSession.builder()
      
